In [1]:
import tensorflow.compat.v2 as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import random
import glob
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import time
keras = tf.keras
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image/255.0
    image = 2*image-1
    return image

In [3]:
train_dir = r'C:\Users\24306\Desktop\datasets\kfold\3\train'
test_dir  = r'C:\Users\24306\Desktop\datasets\kfold\3\test'
checkpoint_path = r'C:\Users\24306\Desktop\RegNetY002\3\RegNetY002.ckpt'

# 处理训练数据

In [4]:
#打乱顺序
train_root = pathlib.Path(train_dir)
train_image_paths = list(train_root.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)
#处理训练标签
train_image_labels = [pathlib.Path(path).parent.name for path in train_image_paths]
train_labels = [[label.split('_')[0]] for label in train_image_labels]
new_train_labels=[]
for n in train_labels:
    new_train_labels.append(float(n[0]));  
train_labels=new_train_labels
print(train_labels)
#处理训练图像
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_label_ds = tf.data.Dataset.from_tensor_slices(train_labels)
train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
train_count = len(train_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
train_data = train_image_label_ds.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(train_data)



[32.0, 86.0, 7.0, 16.0, 37.0, 16.0, 47.0, 40.0, 66.0, 100.0, 20.0, 37.0, 30.0, 45.0, 66.0, 37.0, 37.0, 10.0, 4.0, 47.0, 20.0, 93.0, 7.0, 55.0, 84.0, 30.0, 80.0, 37.0, 35.0, 95.0, 84.0, 16.0, 13.0, 32.0, 37.0, 55.0, 32.0, 93.0, 47.0, 86.0, 47.0, 100.0, 93.0, 35.0, 93.0, 35.0, 10.0, 47.0, 96.0, 10.0, 4.0, 37.0, 37.0, 32.0, 84.0, 52.0, 100.0, 20.0, 66.0, 100.0, 55.0, 47.0, 66.0, 47.0, 96.0, 95.0, 66.0, 47.0, 20.0, 100.0, 95.0, 80.0, 39.0, 100.0, 66.0, 37.0, 4.0, 93.0, 23.0, 95.0, 30.0, 66.0, 47.0, 47.0, 40.0, 47.0, 80.0, 10.0, 7.0, 32.0, 95.0, 47.0, 37.0, 86.0, 20.0, 52.0, 35.0, 96.0, 47.0, 95.0, 7.0, 84.0, 95.0, 93.0, 55.0, 13.0, 37.0, 45.0, 45.0, 84.0, 47.0, 66.0, 13.0, 10.0, 84.0, 37.0, 52.0, 47.0, 84.0, 95.0, 23.0, 100.0, 16.0, 96.0, 95.0, 93.0, 95.0, 37.0, 100.0, 93.0, 30.0, 84.0, 80.0, 30.0, 80.0, 93.0, 95.0, 52.0, 95.0, 30.0, 96.0, 52.0, 84.0, 52.0, 47.0, 37.0, 66.0, 20.0, 84.0, 40.0, 13.0, 100.0, 96.0, 16.0, 37.0, 7.0, 47.0, 40.0, 37.0, 84.0, 37.0, 100.0, 52.0, 35.0, 93.0, 13.0, 2

# 处理验证数据

In [5]:
#打乱顺序
test_root = pathlib.Path(test_dir)
test_image_paths = list(test_root.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
#random.shuffle(train_image_paths)
#处理训练标签
test_image_labels = [pathlib.Path(path).parent.name for path in test_image_paths]
test_labels = [[label.split('_')[0]] for label in test_image_labels]
new_test_labels=[]
for n in test_labels:
    new_test_labels.append(float(n[0]));  
test_labels=new_test_labels
print(test_labels)
#处理训练图像
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
test_image_ds = test_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_labels)
test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
test_count = len(test_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
test_data = test_image_label_ds.shuffle(buffer_size=test_count).repeat(-1)
test_data = test_data.batch(BATCH_SIZE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(test_data)

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.

In [6]:
#要改
RegNetY002 =tf.keras.applications.regnet.RegNetY002(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
#要改
RegNetY002.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
#要改
x = RegNetY002(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out_weight = tf.keras.layers.Dense(1,name='out_weight')(x)

model = tf.keras.Model(inputs=inputs,
                       outputs=[out_weight])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 regnety002 (Functional)     (None, 7, 7, 368)         2814844   
                                                                 
 global_average_pooling2d (G  (None, 368)              0         
 lobalAveragePooling2D)                                          
                                                                 
 out_weight (Dense)          (None, 1)                 369       
                                                                 
Total params: 2,815,213
Trainable params: 2,794,365
Non-trainable params: 20,848
_________________________________________________________________


In [7]:
print ('start time:',time.asctime( time.localtime(time.time()) ))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.01),loss='mse')

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,monitor='val_loss')
history = model.fit(train_data,
                    steps_per_epoch=train_count//BATCH_SIZE,
                    epochs=300,
                    validation_data=test_data,
                    validation_steps=test_count//BATCH_SIZE,
                    callbacks=[cp_callback])
#要改
np.savetxt('RegNetY002_loss_3.txt',history.history['loss'])
#要改
np.savetxt('RegNetY002_valloss_3.txt',history.history['val_loss'])
print ('end time:',time.asctime( time.localtime(time.time()) ))

start time: Fri Apr 14 10:20:07 2023
Epoch 1/300
591/591 [==============================] - ETA: 0s - loss: 433.7913

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY002\3\RegNetY002.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY002\3\RegNetY002.ckpt\assets


591/591 [==============================] - 76s 106ms/step - loss: 433.7913 - val_loss: 10.9628
Epoch 2/300
591/591 [==============================] - ETA: 0s - loss: 12.3864

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY002\3\RegNetY002.ckpt\assets


...

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY002\3\RegNetY002.ckpt\assets


591/591 [==============================] - 102s 173ms/step - loss: 0.4637 - val_loss: 0.2361
Epoch 267/300
591/591 [==============================] - 87s 146ms/step - loss: 0.4433 - val_loss: 0.2871
Epoch 268/300
591/591 [==============================] - 86s 146ms/step - loss: 0.4125 - val_loss: 0.3658
Epoch 269/300
591/591 [==============================] - 88s 149ms/step - loss: 0.4664 - val_loss: 0.3041
Epoch 270/300
591/591 [==============================] - 86s 145ms/step - loss: 0.4271 - val_loss: 0.3089
Epoch 271/300
591/591 [==============================] - 86s 146ms/step - loss: 0.4937 - val_loss: 0.2473
Epoch 272/300
591/591 [==============================] - 85s 144ms/step - loss: 0.4328 - val_loss: 0.3096
Epoch 273/300
591/591 [==============================] - 86s 145ms/step - loss: 0.4933 - val_loss: 0.2633
Epoch 274/300
591/591 [==============================] - 87s 147ms/step - loss: 0.4118 - val_loss: 0.2436
Epoch 275/300
591/591 [==============================] - 86

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY002\3\RegNetY002.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY002\3\RegNetY002.ckpt\assets


591/591 [==============================] - 108s 183ms/step - loss: 0.4444 - val_loss: 0.2331
Epoch 279/300
591/591 [==============================] - 87s 147ms/step - loss: 0.4533 - val_loss: 0.3166
Epoch 280/300
591/591 [==============================] - 88s 149ms/step - loss: 0.3813 - val_loss: 0.2504
Epoch 281/300
591/591 [==============================] - 88s 149ms/step - loss: 0.4329 - val_loss: 0.2611
Epoch 282/300
591/591 [==============================] - 89s 150ms/step - loss: 0.4138 - val_loss: 0.3232
Epoch 283/300
591/591 [==============================] - 90s 152ms/step - loss: 0.4382 - val_loss: 0.3156
Epoch 284/300
591/591 [==============================] - 92s 156ms/step - loss: 0.5082 - val_loss: 0.3742
Epoch 285/300
591/591 [==============================] - 93s 157ms/step - loss: 0.4153 - val_loss: 0.2462
Epoch 286/300
591/591 [==============================] - 94s 159ms/step - loss: 0.4189 - val_loss: 0.2496
Epoch 287/300
591/591 [==============================] - 91